In [67]:
# test
import pandas as pd
import numpy as np

#Export PCAP to CSV
cleaned_PCAP = pd.read_csv('Converted_PCAP.csv')


print(cleaned_PCAP.columns)

cleaned_PCAP['Start_port'] = 0
cleaned_PCAP['End_port'] = 0
cleaned_PCAP['Syn'] = 0
cleaned_PCAP['Seq'] = 0 
cleaned_PCAP['Win'] = 0 
cleaned_PCAP['Len'] = 0 
cleaned_PCAP['MSS'] = 0 

cleaned_PCAP['TCP Retransmission'] = 0
cleaned_PCAP['PSH, ACK'] = 0
cleaned_PCAP['Ack'] = 0
cleaned_PCAP['TSval'] = 0 
cleaned_PCAP['TSecr'] = 0
#59434  >  13530 [SYN] Seq=0 Win=1024 Len=0
#[TCP Retransmission] 33268  >  22 [PSH, ACK] Seq=1 Ack=1 Win=24576 Len=36 TSval=4239907771 TSecr=3409435846

#print(cleaned_PCAP)

Index(['No.', 'Time', 'Source', 'Destination', 'Protocol', 'Length', 'Info'], dtype='object')


In [68]:
import re
for i in range(len(cleaned_PCAP)):
    #print(cleaned_PCAP.loc[i, "Info"])
    info = str(cleaned_PCAP.loc[i, "Info"])
    pattern1 = r"SYN"
    pattern2 = r"PSH, ACK"
    pattern3 = r"TCP Retransmission"
    
    match = re.search(pattern1, info)
    if match:
        cleaned_PCAP.loc[i, "Syn"] = 1
    
    match = re.search(pattern2, info)
    if match:
        cleaned_PCAP.loc[i, "PSH, ACK"] = 1
    
    

    p = re.compile('\d+')
    p = p.findall(info)

    #print(info)
    #print(p)
    match = re.search(pattern3, info)
    if match and len(p) >= 8:
        cleaned_PCAP.loc[i, "TCP Retransmission"] = 1

        #[TCP Retransmission] 33268  >  22 [PSH, ACK] Seq=1 Ack=1 Win=24576 Len=36 TSval=4239907771 TSecr=3409435846
        
        cleaned_PCAP.loc[i, "Start_port"] = p[0]
        cleaned_PCAP.loc[i, "End_port"] = p[1]
        cleaned_PCAP.loc[i, "Seq"] = p[2]
        cleaned_PCAP.loc[i, "Ack"] = p[3]
        cleaned_PCAP.loc[i, "Win"] = p[4]
        cleaned_PCAP.loc[i, "Len"] = p[5]
        cleaned_PCAP.loc[i, "TSval"] = p[6]
        cleaned_PCAP.loc[i, "TSecr"] = p[7]
    elif len(p) >= 5 :
        cleaned_PCAP.loc[i, "Start_port"] = p[0]
        cleaned_PCAP.loc[i, "End_port"] = p[1]
        cleaned_PCAP.loc[i, "Seq"] = p[2]
        cleaned_PCAP.loc[i, "Win"] = p[3]
        cleaned_PCAP.loc[i, "Len"] = p[4]
    else:
        cleaned_PCAP.loc[i, "Start_port"] = -1
        
    
        


In [69]:

cleaned_PCAP = cleaned_PCAP[cleaned_PCAP['Start_port'] != -1]
cleaned_PCAP = cleaned_PCAP.drop(['No.', 'Source', 'Destination'], axis=1)


In [70]:
one_hot = pd.get_dummies(cleaned_PCAP['Protocol'])
# Drop column B as it is now encoded
cleaned_PCAP = cleaned_PCAP.drop('Protocol',axis = 1)
# Join the encoded df
cleaned_PCAP = cleaned_PCAP.join(one_hot)

In [71]:
cleaned_PCAP.to_csv("cleaned_PCAP.csv",  encoding='utf-8')